In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from goto import with_goto
import os
import cmath
from scipy import stats
import numpy.linalg as lin
import time

In [3]:
plt.style.use('ggplot')
%matplotlib qt

### clg, clog, ce, cexp, sm, sm_, smu bunlar benim yazdığım gauss'da olan fakat python karşılığı bulunmayan fonksiyonlar. 

In [123]:
clg = lambda x: cmath.log(x)

In [124]:
clog = np.vectorize(clg)

In [134]:
ce = lambda x: cmath.exp(x)

In [135]:
cexp = np.vectorize(ce)

In [4]:
small_con = lambda x: 1 if bool(x == True) else 0
sm = np.vectorize(small_con)

In [5]:
sm1 = lambda x: int(x)
sm_ = np.vectorize(sm1)

In [6]:
def smu(b,s):
    v = np.zeros((np.shape(b)[0],np.shape(b)[1]))
    for i in range(0,np.shape(b)[1]):
        for j in range(0,np.shape(b)[0]):
            if b[j,i] and s[i]:
                v[j,i] = 1
            else:
                v[j,i] = 0
    return v

### function: parameter update for cost function and its derivatives

In [7]:
def function_():
    
    #PHASE 1: UPDATE PARAMETERS CORRESPONDING TO CHOSEN MODEL
    
    global bet,tr,c,dc
    global p, lnp, invp
    
    tr = q[0]
    c1 = q[1]
    
    if (modnum[[1,3]].flatten() == np.array([[2,0]])).all() :
        c2 = q[2]
    
    elif (modnum[[1,3]].flatten() == np.array([[3,0]])).all() :
        c2 = q[2]
        c3 = q[3]
    
    elif (modnum[[1,3]].flatten() == np.array([[4,0]])).all() :
        c2 = q[2]
        c3 = q[3]
        c4 = q[4]
        
    elif (modnum[[1,3]].flatten() == np.array([[1,1]])).all() :
        bet = 1/(1+np.exp(q[2]))
        
    elif (modnum[[1,3]].flatten() == np.array([[2,1]])).all() :
        c2 = q[2]
        bet = 1/(1+np.exp(q[3]))
        
    elif (modnum[[1,3]].flatten() == np.array([[3,1]])).all() :
        c2 = q[2]
        c3 = q[3]
        bet = 1/(1+np.exp(q[4]))

    elif (modnum[[1,3]].flatten() == np.array([[4,1]])).all() :
        c2 = q[2]
        c3 = q[3]
        c4 = q[4]
        bet = 1/(1+np.exp(q[5]))
        
    if int(modnum[2]) != 1:
        p[0:int(modnum[4]),0] = q[(dm-int(modnum[4])-int(modnum[6])):(dm-int(modnum[6])),0]
    
    #PHASE 2: UPDATE TRANSITION PROBABILITIES FOR FULL LIKELIHOOD
    
    lnp = np.log(p)
    invp = np.zeros((int(modnum[4]),int(modnum[4])))
    np.fill_diagonal(invp,np.exp(-lnp[0:int(modnum[4])]))
    invp = np.concatenate([invp,-np.exp(-lnp[int(modnum[4])])*np.ones((1,int(modnum[4])))], axis=0)
    
    #EVALUATE c(x) AND DERIVATIVE WRT (c1,..,c4) AT NEW PARAMETERS
    
    if modnum[1] == 2:
        
        if modnum[0] == 1:
            
            c = 0.01*c1*ogrid+(0.0001*c2*(ogrid**2))
            dc = np.concatenate([(0.01*ogrid),(0.0001*(ogrid**2))],axis=1)
        
        elif modnum[0] == 2:
            
            c = (0.001*c1*ogrid)+(0.000001*c2*(ogrid**2))
            dc = np.concatenate([(0.001*ogrid), (0.000001*(ogrid**2))], axis=1)
        
        elif modnum[0] == 3:
            
            c = c1*(ogrid**(0.001*c2))
            dc = np.concatenate([c/c1, (0.001*np.log(ogrid)*c)],axis=1)
        
        elif modnum[0] == 4:
            
            c  = c2/ogrid+0.01*c1*(ogrid**(0.5))
            dc = np.concatenate([(0.01*(ogrid**(0.5))), (1/ogrid)],axis=1)
        
        elif modnum[0] == 5:
            
            c = c2/(ogrid**(0.5)) + c1*(ogrid**(0.5))
            dc = np.concatenate([(ogrid**(0.5)), (1/(ogrid**(0.5)))], axis=1)
        
        elif modnum[0] == 6:
            
            c = c2/ogrid**(2)+c1*ogrid**(0.5)
            dc = np.concatenate([(ogrid**(0.5)), (1/(ogrid**(2)))], axis=1)
        
        elif modnum[0] == 7:
            
            c = c1*np.exp(0.001*c2*ogrid)
            dc = np.concatenate([(c/c1), (0.001*ogrid*c)], axis=1)
    
   
    if modnum[1] == 1:
        
            if modnum[0] == 1:
                
                c = c1*ogrid
                dc = ogrid
            
            elif modnum[0] == 2:
                
                c = 0.001*c1*ogrid
                dc = 0.001*ogrid
            
            elif modnum[0] == 3:
                
                c = c1*(ogrid**(0.5))
                dc = ogrid**(0.5)
            
            elif modnum[0] == 4:
                
                c = c1/ogrid
                dc = 1/(ogrid)
            
            elif modnum[0] == 5:
                
                c = 0.001*c1*(ogrid**(0.5))
                dc = c/c1
                
       
    if modnum[1] == 3:
        
            if modnum[0] == 1:
                
                c = 0.01*c1*ogrid+0.0001*c2*(ogrid**(2))+0.000001*c3*(ogrid**(3))
                dc = np.concatenate([(0.01*ogrid), (0.0001*ogrid**(2)), (0.000001*ogrid**(3))], axis=1)
            
            elif modnum[0] == 2:
                
                c = c1*ogrid*(sm(ogrid<150))+(c1*150+c2*(ogrid-150)) * (sm(150<=ogrid)) * (sm(ogrid<300)) + (((c1+c2)*150+c3*(ogrid-300))*(sm(300<=ogrid)))
                dc = np.concatenate([(ogrid*(sm(ogrid<150))+150*(sm(ogrid>=150))), ((ogrid-150)*(sm(ogrid>=150))*(sm(ogrid<300))+150*(sm(ogrid>=300))), ((ogrid-300)*(sm(ogrid>=300)))], axis=1)
            
    
    elif modnum[1] == 4:    
        
            if modnum[0] == 1:
                
                c = 0.01*c1*ogrid+0.0001*c2*ogrid**(2)+0.000001*c3*ogrid**(3)+0.000000001*c4*ogrid**(4)
                dc = np.concatenate([(0.01*ogrid), (0.0001*ogrid**(2)), (0.000001*ogrid**(3)),(0.00000001*ogrid**(4))], axis=1)
            
            else:
                
                c = c1*ogrid*(sm(ogrid<100))+((c1*100+c2*(ogrid-100))*(sm(100<=ogrid))*(sm(ogrid<200)))+(((c1+c2*100+c3*(ogrid-200))*(sm(200<=ogrid))*(sm(ogrid<300)))+(((c1+c2+c3)*100+c4*(ogrid-300))*(sm(300<=ogrid))))         
                dc = np.concatenate([(ogrid*(sm(ogrid<100))+100*(sm(100<=ogrid))),((ogrid-100)*(sm(100<=ogrid))*(sm(ogrid<200))+100*(sm(ogrid>=200))),((ogrid-200)*(sm(ogrid>=200))*(sm(ogrid<300))+100*(sm(ogrid>=300))),((ogrid-300)*(sm(ogrid>=300)))], axis=1)
  


### eval: evaluation of likelihood function and its derivatives

In [8]:
@with_goto
def eval_():
    
    global lp, d1v, u, dll, ll
    
    #EVALUATE PROBABILITY OF KEEPING BUS ENGINE AT EACH SAMPLE POINT
    lp = pk[dt[:,1]-1,:]
    
    if i < 0:
        
        goto .bypass


    #COMPUTE DERIVATIVES OF LOGLIKELIHOOD FUNCTION
    
    #CASE 1: ESTIMATE FULL LIKELIHOOD BUT NOT bet
    
    if (modnum[[2,3]].flatten() == np.array([[2,0]])).all():
            
        d1v = np.concatenate([(-np.ones(nt,1)), dc[dt[:,1]-1,:], np.zeros(nt, modnum[4]) ])
        
    
    #CASE 2: ESTIMATE FULL LIKELIHOOD AND BET
    
    elif (modnum[[2,3]].flatten() == np.array([[2,1]])).all():
        
        d1v = np.concatenate([(-np.ones((nt,1))), (dc[dt[:,1]-1,:]), (-bet*(1-bet)*(ev[0,0]-ev[dt[:,1]-1,:])),
                              np.zeros((nt,int(modnum[4])))], axis=1)        
        
    
    
    #CASE 3: ESTIMATE PARTIAL LIKELIHOOD BUT NOT bet
    
    elif (modnum[[2,3]].flatten() == np.array([[1,0]])).all():
    
        d1v = np.concatenate([-np.ones((nt,1)), dc[dt[:,1]-1,:]], axis=1)
        
   
    #CASE 4: ESTIMATE PARTIAL LIKELIHOOD AND bet
    
    elif (modnum[[2,3]].flatten() == np.array([[1,1]])).all():
        
        d1v = np.concatenate([-np.ones((nt,1)), dc[dt[:,1]-1,:], (-bet*(1-bet)*(ev[0,0]-ev[dt[:,1]-1,:]))])
    
    
    #ADD ON DERIVATIVES OF EXPECTED VALUE FUNCTION, BYPASS IF bet=0 
    
    if bet == 0:
        
        d1v = d1v *(lp-1+dt[:,0].reshape(dt.shape[0],1))            
        
        if modnum[2]==2:
            
            g = dm - int(modnum[4]) + 1
            
            d1v[:,(g-1):dm] = invp[dt[:,2],:]
            
        
        goto .cumulate
        
   
    #FIRST TWO DERIVATIVES (tr,c1) ARE SAME FOR ALL MODELS
    
    d1v[:,0:2] = (d1v[:,0:2]+dev[0,0:2]*np.ones((nt,1))-dev[:,0:2][dt[:,1]-1,:])*(lp-1+dt[:,0].reshape(dt.shape[0],1))
    
    
    
    #DERIVATIVES FOR SUMMANDS OF PARTIAL LOGLIKELIHOOD FUNCTION
    
    if modnum[2] == 1:
    
        for g in range(3,dm+1):

            u = (np.array([d1v[:,(g-1)].T + dev[0,(g-1)]]).T * np.ones((nt,1))-np.array([dev[:,(g-1)]]).T[dt[:,1]-1,:]) * (lp-1+dt[:,0].reshape(dt.shape[0],1))
            d1v[:,(g-1)] = u.flatten()
    
    
    #DERIVATIVES FOR SUMMANDS OF FULL LOGLIKELIHOOD FUNCTION 
    
    elif modnum[2] == 2:
            
         for g in range(3,dm-int(modnum[4])+1):

            u = (np.array([d1v[:,(g-1)] + dev[0,(g-1)]]).T * np.ones((nt,1))-np.array([dev[:,(g-1)]]).T[dt[:,1]-1,:]) * (lp-1+dt[:,0].reshape(dt.shape[0],1))
            d1v[:,(g-1)] = u.flatten()
        
        
         g = dm - int(modnum[4]) + 1
        
         d1v[:,(g-1):dm]=((dev[0,(g-1):dm]*np.ones((nt,1))-dev[:,(g-1):dm][dt[:,1]-1,:]))*(lp-1+dt[:,0].reshape(dt.shape[0],1)) + invp[dt[:,2],:]
        
    
    label .cumulate
    
    dll = np.array([d1v.sum(axis=0)]).T + dll
    
    label .bypass
    
    
    #FULL LOGLIKELIHOOD CUMULATION 

    if modnum[2] == 2:

        ll =  ll + np.array([np.sum(np.array([list(map(cmath.log,(lp+(1-2*lp)*np.array([dt[:,0]]).T)))]).T +  lnp[dt[:,2],:], axis=0)]).T


    #PARTIAL LOGLIKELIHOOD CUMULATION 

    if modnum[2] == 1:   
        
        ll =  ll + np.array([np.sum(np.array([list(map(cmath.log,(lp+(1-2*lp)*np.array([dt[:,0]]).T)))]).T, axis=0)]).T


    
   

### eval1:  evaluation of likelihood function and its derivatives EVAL1 differs from EVAL by the inclusion of the lagged dependent variable i(t-1)

In [9]:
@with_goto
def eval1_():
    
    global d1v, lp, ll, u, dll
    
    #EVALUATE PROBABILITY OF KEEPING BUS ENGINE AT EACH SAMPLE POINT

    if bet > 0:
        lp1 = (1/(1+np.exp(c-bet*ev-tr-c[0,0]+bet*ev[0,0]+q[(dm-1),0])))[dt[:,1]-1,:]
        
    else:
        lp1 = (1/(1+np.exp(c-tr-c[0,0]+q[dm-1,0])))[dt[:,1]-1,:]
        
    lp = pk[dt[:,1]-1,:]
    
    if i < 0:
        
        goto .loglik
        
        
    #COMPUTE DERIVATIVES OF LOGLIKELIHOOD FUNCTION
    
    
    #CASE 1: ESTIMATE FULL LIKELIHOOD BUT NOT bet
    
    if (modnum[0,2:4].flatten() == np.array([[2,0]])).all():
        
        d1v = np.concatenate([-np.ones((nt,1)), dc[dt[:,1]-1,:], np.zeros((nt, modnum[0,4]))], axis=1)
    
    
    #CASE 2: ESTIMATE FULL LIKELIHOOD AND bet
        
    elif (modnum[0,2:4].flatten() == np.array([[2,1]])).all():
        
        d1v = np.concatenate([(-np.ones((nt,1))), (dc[dt[:,1]-1,:]), (-bet*(1-bet)*(ev[0,0]-ev[dt[:,1]-1])), np.zeros((nt, modnum[0,4]))], axis=1)
        
   
    #CASE 3: ESTIMATE NEITHER PARTIAL LIKELIHOOD NOR bet
    
    elif (modnum[0,2:4].flatten() == np.array([[1,0]])).all():
        
        d1v = np.concatenate([(-np.ones((nt,1))), dc[dt[:,1]-1,:],], axis=1)
        
    
    #CASE 4: ESTIMATE PARTIAL LIKELIHOOD AND bet
    
    elif (modnum[0,2:4].flatten() == np.array([[1,1]])).all():
        
        d1v = np.concatenate([(-np.ones((nt,1))), (dc[dt[:,1]-1,:]),(-bet*(1-bet)*(ev[0,0]-ev[dt[:,1]-1,:]))], axis=1 )                 
        
    

    #CUMULATE LOGLIKELIHOOD FUNCTION

    
    #FULL LOGLIKELIHOOD FUNCTION
    
    label .loglik

    if modnum[0,2] == 2:
        
        ll = ll + np.array([np.sum(np.vectorize(cmath.log)(((lp+(1-2*lp)*np.array([dt[:,0]]).T)*np.array([dt[:,3]]).T+(lp1+(1-2*lp1)*np.array([dt[:,0]]).T)*(1-np.array([dt[:,3]]).T)))+ lnp[dt[:,2],:], axis=0)]).T

    
    #PARTIAL LOGLIKELIHOOD FUNCTION
    
    else:
        
        ll = ll + np.array([np.sum(np.vectorize(cmath.log)(((lp+(1-2*lp)*np.array([dt[:,0]]).T)*np.array([dt[:,3]]).T+(lp1+(1-2*lp1)*np.array([dt[:,0]]).T)*(1-np.array([dt[:,3]]).T))), axis=0)]).T
        
    
    if i < 0:
        
        goto .bypass
        
    lp1 = ((lp-1+np.array([dt[:,0]]).T)*np.array([dt[:,3]]).T+(lp1-1+np.array([dt[:,0]]).T)*np.array([1-dt[:,3]]).T) 
    
    
    #ADD ON DERIVATIVES OF EXPECTED VALUE FUNCTION, BYPASS IF bet=0
    
    if bet == 0:
        
        d1v = d1v * lp1
    
        if modnum[0,2] == 2:
            
            g = dm - modnum[0,4]
            d1v[:,g-1:dm-1] = invp[dt[:,2],:]
            
            
        goto .cumulate
        
        
    #FIRST TWO DERIVATIVES (tr,c1) ARE SAME FOR ALL MODELS

    #In Gauss, I removed a column from lp1 to make sure the matrices are comfortable and then checking if the outputs are the same.

    d1v[:,0:2]=(d1v[:,0:2]+dev[0,0:2] * np.ones((nt,1))-dev[:,0:2][dt[:,1]-1,:])*lp1


    #DERIVATIVES FOR SUMMANDS OF PARTIAL LOGLIKELIHOOD FUNCTION
    
    if modnum[0,2] == 1:
        
        for g in range(3, dm):
            u = (((np.array([d1v[:,(g-1)]]).T) + (np.array([dev[0,(g-1)]]).T)*np.ones((nt,1)))-(np.array([dev[:,(g-1)]]).T)[dt[:,1]-1,:])*lp1
            d1v[:,(g-1)] = u.flatten()
            
    
    #DERIVATIVES FOR SUMMANDS OF FULL LOGLIKELIHOOD FUNCTION
    
    elif modnum[0,2] == 2:
        
        for g in range(3,dm-modnum[0,4]):
            u = (d1v[:,(g-1)]+ dev[0,(g-1)]*np.ones(nt,1)-dev[:,(g-1)][dt[:,1]-1,:])*lp1
            d1v[:,(g-1)] = u.flatten()
        
        
        g = dm-int(modnum[0,4])
        d1v[:,(g-1):(dm-1)]=(dev[0,(g-1):(dm-1)]*np.ones((nt,1))-dev[:,(g-1):dm-1][dt[:,1]-1,:])*lp1+invp[dt[:,2],:]
    
    
    
    
    #CUMULATE DERIVATIVES OF LOGLIKELIHOOD FUNCTION
    
    label .cumulate

    d1v = np.concatenate([d1v, ((1-np.array([dt[:,3]])).T*lp1)], axis=1)

    dll = dll + np.array([np.sum(d1v, axis=0)]).T
    
    
    label .bypass
            

### cdtp:  computation of derivatives of contraction operator with respect to transition probability parameters

In [10]:
def cdtp(z): 
    
    global dtp
    dtp = np.log(np.exp(-c+bet*z)+np.exp(-tr-c[0,0]+bet*z[0,0]))

    if modnum[0,4] == 2:

        dtp = np.concatenate([np.concatenate([np.array([(dtp[0:n-2,0]-dtp[2:n, 0])]).T, np.array([[dtp[n-2,0]-dtp[n-1,0]]]), np.array([[0]])], axis=0),
                            np.concatenate([np.array([dtp[1:n-1,0]-dtp[2:n,0]]).T, np.array([[0]]), np.array([[0]])], axis=0) ], axis=1)

    elif modnum[0,4] == 1:

        dtp = np.concatenate([np.array([(dtp[0:n-1,0])-dtp[1:n,0]]).T, np.array([[0]])], axis=0)

    elif modnum[0,4] == 3:

        dtp = np.concatenate([ np.concatenate([np.array([(dtp[0:n-3,0]-dtp[3:n,0])]).T,np.array([[dtp[n-3,0]-dtp[n-1,0]]]),np.array([[dtp[n-2,0]-dtp[n-1,0]]]), np.array([[0]])], axis=0),
                   np.concatenate([np.array([dtp[1:n-2,0]-dtp[3:n,0]]).T, np.array([[dtp[n-2,0]-dtp[n-1,0]]]), np.array([[0],[0]])]),
                   np.concatenate([np.array([dtp[2:n-1,0]-dtp[3:n,0]]).T, np.array([[0],[0],[0]])])], axis=1)

    elif modnum[0,4] == 4:

        dtp = np.concatenate([np.concatenate([np.array([(dtp[0:n-4,0]-dtp[4:n,0])]).T,np.array([[dtp[n-4,0]-dtp[n-1,0]]]), np.array([[dtp[n-3,0]-dtp[n-1,0]]]), np.array([[dtp[n-2,0]-dtp[n-1,0]]]),np.array([[0]])], axis=0),
                   np.concatenate([np.array([dtp[1:n-3,0]-dtp[4:n,0]]).T, np.array([[dtp[n-3,0]-dtp[n-1,0]]]),np.array([[dtp[n-2,0]-dtp[n-1,0]]]),np.array([[0],[0]])], axis=0),
                   np.concatenate([np.array([dtp[2:n-2,0]-dtp[4:n,0]]).T, np.array([[dtp[n-2,0]-dtp[n-1,0]]]), np.array([[0],[0],[0]])],axis=0 ),
                   np.concatenate([np.array([dtp[3:n-1,0]-dtp[4:n,0]]).T, np.array([[0],[0],[0],[0]])],axis=0)], axis=1)


    elif modnum[0,4] == 5:

        dtp = np.concatenate([np.concatenate([np.array([dtp[0:n-5,0]-dtp[5:n,0]]).T, np.array([[dtp[n-5,0]-dtp[n-1,0]]]),np.array([[dtp[n-4,0]-dtp[n-1,0]]]),np.array([[dtp[n-3,0]-dtp[n-1,0]]]),np.array([[dtp[n-2,0]-dtp[n-1,0]]]),np.array([[0]])], axis=0),
                   np.concatenate([np.array([dtp[1:n-4,0]-dtp[5:n,0]]).T, np.array([[dtp[n-4,0]-dtp[n-1,0]]]),np.array([[dtp[n-3,0]-dtp[n-1,0]]]),np.array([[dtp[n-2,0]-dtp[n-1,0]]]),np.array([[0],[0]])], axis=0),
                   np.concatenate([np.array([dtp[2:n-3,0]-dtp[5:n,0]]).T,np.array([[dtp[n-3,0]-dtp[n-1,0]]]),np.array([[dtp[n-2,0]-dtp[n-1,0]]]),np.array([[0],[0],[0]])], axis=0),
                   np.concatenate([np.array([dtp[3:n-2,0]-dtp[5:n,0]]).T, np.array([[dtp[n-2,0]-dtp[n-1,0]]]), np.array([[0],[0],[0],[0]])],axis=0),
                   np.concatenate([np.array([dtp[4:n-1,0]-dtp[5:n,0]]).T,np.array([[0],[0],[0],[0],[0]])], axis=0)], axis=1)

### center: procedure for centering value function about 0

In [11]:
def center(z):
    global c
    y1 = -np.array([np.mean(c, axis=0)]).transpose() + (bet * np.array([np.mean(z, axis=0)]).transpose())  
    c = c + y1 * (1-bet)
    return(z-y1)

### e: conditional expectation operator for Markov transition matrix

In [97]:
def e(y):
    
    global p
    
    m = int(modnum[4])
    r = np.shape(y)[1]
    
    
    y1 = p[0]*y[0:n-m, ]
    y2 = np.zeros((m,1))
    y2[m-1,] = y[n-1,]
        
    for i in range(1, m+1):
      
        y1=y1+p[i]*y[i:n-m+i,]
                
        for j in range(1,m):
            
            sp = 0
            y3 = np.zeros((1,r))            
            
            for k in range(1,m-j+1):
                
                sp = sp + p[k-1]
                y3 = y3 + p[k-1]*y[n-m+k+(j-1)-1,]
                
            
            y3 = y3 + (1-sp) * y[n-1,]
            
            y2[j-1,] = y3
            
                
    return(np.concatenate((y1,y2),axis=0))



### contract: procedure to compute contraction operator

In [139]:
def contract(x):
    global ev, bet, c, tr
    zmax = (-c+bet*x).max(axis=0)
    y4 = np.exp(-tr-c[0,0]+bet*x[0,0]-zmax) 
    return(zmax+e(clog(cexp(-c+bet*x-zmax)+y4)))

### eq11: parameter update of markov transition probability matrix Q11

In [14]:
def eq11():
    
    global q11
    
    m = math.floor(n/2)
    q11 = np.zeros((m, m + int(modnum[4])))
    for l in range(1,m+1):
        
        q11[l-1, (l-1):(l+int(modnum[4]))]=p.transpose()  
           
    return(q11)
    

### eq22: parameter update of markov transition probability matrix Q22

In [96]:
def eq22():
    
    global q22
    
    m = math.floor(n/2)
    q22 = np.zeros((n-m,n-m))
    for l in range(1,m+1):
        
        
        if l < m-modnum[4]+1:
            q22[l-1, (l-1):l+int(modnum[4])] = p.transpose()
        
        elif l < m:
            t1 = p[0:(m-l)].transpose()
            t2 = np.array([1-np.sum(p[0:m-l], axis=0)])
            q22[l-1,(l-1):m] = np.concatenate([t1,t2], axis=1)
        
        else:
            q22[l-1,l-1]=1
    
    if n > 2*m:
        q22[m,m] = 1
        q22[m-1, (m-1):(m+1)] = np.concatenate([p[0], (1-np.sum(p[0], axis=0))], axis=1)
    
    return(q22)
            

### partsolv:  solution of linear systems by partitioned elimination

In [16]:
def partsolv(z):
    
    global l, t, m, bet, pk, n
    
    
    y1 = z[0:m:,]
    y2 = z[m:n,]
    cy = np.shape(z)[1]
    cq = int(modnum[4])
    q21 = lin.solve((np.eye(n-m)-(q22*(bet*pk[m:n,0]).transpose())),np.concatenate([y2,(np.array([bet-np.sum((q22*(bet*pk[m:n,0]).transpose()).transpose(),axis=0)]).transpose())],axis=1))
    q12 = np.dot((q11[:,m:m+cq]*(bet*pk[m:m+cq,0]).transpose()), q21[0:int(cq)])
    l = np.array([bet - np.sum((q11[:,1:int(m+cq)]*(bet*pk[1:int(m+cq),0]).transpose()).transpose(),axis=0)]).T
    t = np.concatenate([np.array([q12[:,cy]]).transpose(),q11[:,1:m]*(bet*pk[1:m,0])], axis=1)
    t[:,0] = l[:,0] + t [:,0]
    y1 = lin.solve((np.eye(m)-t),(y1+q12[:,0:cy]))
    y2 = q21[:,0:cy]+np.array([q21[:,0+cy]]).T*y1[0,0:cy]
    return np.concatenate([y1,y2],axis=0)
    
    

### ffxp: contraction mapping fixed point polyalgorithm

In [106]:
@with_goto
def ffxp():
    
    global pr, swj, pk, dev, ss2, m, ev, ev1
    
    
    if pr:
    
        print("begin contraction iterations")
    
    nwt = 0
    j = 0 
    tol1 = 1

    
    label .begin

    j = j +2 
    ev1 = contract(ev)
    ev = contract(ev1)
    
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T
    mm = tol/tol1
    tol1 = tol
    
    if pr == 1:
        
        print(tol, mm, j)
    
    if j >= cstp and nstep == 1:
        
        swj = j
        nwt = 0
        
        if pr == 0:
            
            print(tol, mm, j)
    
        if pr:

            print('begin Newton-Kantorovich iterations at time: {}'.format(time.asctime( time.localtime(time.time()) )))

        ev1 = contract(ev)
        
        goto .fin
        
    
    if tol < 0.1 or j >= cstp:
        
        goto .newt
    
    if j >= mincstp and mm > bet*bet-rtol:
        
        goto .newt
    
    goto .begin
        
    #PHASE 2: NEWTON-KANTOROVICH ITERATIONS
    
    label .newt
    
    if pr == 2:
        
        print(tol, mm, j)
    
    nwt = 1
    swj = j
    
    if pr:
        print('begin Newton-Kantorovich iterations at time: {}'.format(time.asctime( time.localtime(time.time()) )))
    
    ev1 = contract(ev)
    
    
    label .qmat
    
    pk = 1/(1+np.exp(c-bet*(d*ev+(1-d)*ev1)-tr-c[0,0]+bet*(d*ev[0,0]+(1-d)*ev1[0,0])))
    
    ev1 = ev-partsolv((ev-ev1))
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T

    ev = contract(ev1)
    
    if pr:
        print(tol)
    
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T

    if pr:
        print(tol)
        
    if tol < ltol or nwt > nstep:
        
        goto .fin1
    
    nwt = nwt + 1
    
    pk = 1/(1+np.exp(c-bet*(d*ev1+(1-d)*ev)-tr-c[0,0]+bet*(d*ev1[0,0]+(1-d)*ev[0,0])))
    ev = ev1-partsolv((ev1-ev))
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T
    
     
    
    if pr:
        print(tol)
        tol
    
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T
    
    if pr:
        print(tol)
    
    if tol < ltol or nwt > nstep:
        
        goto .fin
    
    else:
        nwt = nwt + 1
        goto .qmat
    
  
    #PHASE 3: FINAL N-K STEP AND CALCULATION OF DERIVATIVES OF EV
    
    label .fin
    
    nwt = nwt + 1
    pk = 1/(1+np.exp(c-bet*(d*ev+(1-d)*ev1)-tr-c[0,0]+bet*(d*ev[0,0]+(1-d)*ev1[0,0])))
    
    if modnum[2] == 1:
        
        goto .dv
    
    cdtp(ev)
    print(cdtp(ev))
    
    label .dv
    
    dev = np.concatenate([(e(pk)-1), (e(((dc[0,:]-dc)*pk))-dc[0,:])], axis=1)
    
    if (modnum[[2,3]].flatten() == np.array([[2,1]])).all():
        
        dev = np.concatenate([dev, (-bet*(1-bet)*(ev[0,0]+e(((ev-ev[0,0])*pk)))), dtp, (ev-ev1)], axis=1)
    
    
    elif (modnum[[2,3]].flatten() == np.array([[2,0]])).all():
        
        dev = np.concatenate([dev, dtp, (ev-ev1)], axis=1)

    
    elif (modnum[[2,3]].flatten() == np.array([[1,0]])).all():
        
        dev = np.concatenate([dev, (ev-ev1)], axis=1)
        

    elif (modnum[[2,3]].flatten() == np.array([[1,1]])).all():
        
        dev = np.concatenate([dev, (-bet*(1-bet)*(ev[0,0]+e(((ev-ev[0,0])*pk)))), (ev-ev1)], axis=1)
    
    dev = partsolv(dev)
    ev1 = ev-np.array([dev[:,dm-int(modnum[6])]]).T
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T
    dev[:,0:dm-int(modnum[6])] = bet*dev[:,0:dm-int(modnum[6])]
    
    ev = contract(ev1)
    
    pk = 1/(1+np.exp(c-bet*ev-tr-c[0,0]+bet*ev[0,0]))
    print(tol)
    tol
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T
    print(tol)
    tol
    
    goto .term
    
    
    label .fin1
    
    nwt = nwt + 1
    pk = 1/(1+np.exp(c-bet*(d*ev1+(1-d)*ev)-tr-c[0,0]+bet*(d*ev1[0,0]+(1-d)*ev[0,0])))
    
    if modnum[2] == 1:
        
        goto .dv1
    
    print(cdtp(ev1))
    cdtp(ev1)
    
    
    label .dv1
    
    dev = np.concatenate([(e(pk)-1), (e(((dc[0,:]-dc)*pk))-dc[0,:])], axis=1)
    
    if (modnum[[2,3]].flatten() == np.array([[2,1]])).all():
        
        dev = np.concatenate([dev, (-bet*(1-bet)*(ev1[0,0]+e(((ev1-ev1[0,0])*pk)))), dtp, (ev1-ev)], axis=1)
        
    
    elif (modnum[[2,3]].flatten() == np.array([[2,0]])).all():
        
        dev = np.concatenate([dev, dtp, (ev1-ev)], axis=1)
    
    
    elif (modnum[[2,3]].flatten() == np.array([[1,0]])).all():
        
        dev = np.concatenate([dev, (ev1-ev)], axis=1)
    
    
    elif (modnum[[2,3]].flatten() == np.array([[1,1]])).all():
        
        dev = np.concatenate([dev,(-bet*(1-bet)*(ev1[0,0]+e(((ev1-ev1[0,0])*pk)))),(ev1-ev)],axis=1)

    
    dev = partsolv(dev)
    ev = ev1-dev[:,dm-int(modnum[6])]
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T
    dev[:,0:dm-int(modnum[6])] = bet*dev[:,0:dm-int(modnum[6])]
    
    ev1 = contract(ev)
    
    pk = 1/(1+exp(c-bet*ev1-tr-c[1,1]+bet*ev1[1,1]))
    
    if pr:
        print(tol)
        tol
    
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T
    
    if pr:
        print(tol)
        tol
    
    ev = ev1
    
    
    label .term
    
    if pr:
        
        print('Fixed point time: {}'.format(time.asctime( time.localtime(time.time()) )))

        
    

### stordat: creates bus data file for use by nfxp

In [18]:
@with_goto
def stordat():
    
    global lfe,rws,cstp,modnum,dt,ov1,ov2,nr,m,dtc,mil,n,p,omax,ogrid, dt
    global strng,rt,nt,rc,nc,ll,loop,milecnt,dtx,npk,lp,g,f1,grf, bet,len_
    
    
    lfe = 0
    rws = 0
    cstp = 0
    modnum = np.zeros((7,1))
    
    #SELECT FIXED POINT DIMENSION AND SIZE OF MILEAGE CELLS
    
    n = int(input("enter desired fixed point dimension 'n' "))
    omax = int(input("enter upper bound odometer value (recommend 450,000)"))
    print("implied size of discrete mileage range={}".format(omax/n))
    modnum[6] = int(input("include lagged replacement choice i(t-1) in data set? (1=yes, 0=no)"))
    
    rt = np.zeros((n,1))
    nt = np.zeros((n,1))
    rc = np.zeros((n,1))
    nc = np.zeros((n,1))
    ll = 0
    loop = 1
    milecnt = np.zeros((n,1))
    
    #SELECT BUS GROUPS TO BE INCLUDED IN ESTIMATION SAMPLE
    
    print("enter bus groups to be included in BDT.DAT (1=include, 0=leave out)")
    
    label .start
    
    if loop == 1:
        loop = 2
        swj = int(input("Bus group 1: 1983 Grumman model 870 buses (15 buses total)\n(1=include, 0=leave out)"))
        
        if swj == 1:
            dt = open("g870.asc").read().split()
            for i in range(0,len(dt)):
                dt[i] = int(dt[i])
            dt = np.array(dt)
            dt = dt.reshape((15,np.shape(dt)[0]//15)).T
            goto .dtstor
        else:
            goto .start
    
    
    elif loop == 2:
        loop = 3
        swj = int(input("Bus group 2: 1981 Chance RT-50 buses (4 buses total)\n(1=include, 0=leave out)"))
        
        if swj == 1:
            dt = open("rt50.asc").read().split()
            for i in range(0,len(dt)):
                dt[i] = int(dt[i])
            dt = np.array(dt)
            dt = dt.reshape((4,np.shape(dt)[0]//4)).T
            goto .dtstor
        else:
            goto .start
    
    
    elif loop == 3:
        loop = 4
        swj = int(input("Bus group 3: 1979 GMC model t8h203 buses (48 buses total)\n(1=include, 0=leave out)"))
        
        if swj == 1:
            dt = open("t8h203.asc").read().split()
            for i in range(0,len(dt)):
                dt[i] = int(dt[i])
            dt = np.array(dt)
            dt = dt.reshape((48,np.shape(dt)[0]//48)).T    
            goto .dtstor
        else:
            goto .start
    
    
    elif loop == 4:
        loop = 5
        swj = int(input("Bus group 4: 1975 GMC model a5308 buses (37 buses total)\n(1=include, 0=leave out)"))
        
        if swj == 1:
            dt = open("a530875.asc").read().split()
            for i in range(0,len(dt)):
                dt[i] = int(dt[i])
            dt = np.array(dt)
            dt = dt.reshape((37,np.shape(dt)[0]//37)).T
            goto .dtstor
        else:
            goto .start
    
    
    elif loop == 5:
        loop = 6
        swj = int(input("Bus group 5: 1972 GMC model a5308 buses (18 buses total)\n(1=include, 0=leave out)"))
        
        if swj == 1:
            dt = open("a530872.asc").read().split()
            for i in range(0,len(dt)):
                dt[i] = int(dt[i])
            dt = np.array(dt)
            dt = dt.reshape((18,np.shape(dt)[0]//18)).T 
            goto .dtstor
        else:
            goto .start
            
    
    elif loop == 6:
        loop = 7
        swj = int(input("Bus group 6: 1972 GMC model a4523 buses (18 buses total)\n(1=include, 0=leave out)"))
        
        if swj == 1:
            dt = open("a452372.asc").read().split()
            for i in range(0,len(dt)):
                dt[i] = int(dt[i])
            dt = np.array(dt)
            dt = dt.reshape((18,np.shape(dt)[0]//18)).T 
            goto .dtstor
        else:
            goto .start
    

    elif loop == 7:
        loop = 8
        swj = int(input("Bus group 7: 1974 GMC model a4523 buses (10 buses total)\n(1=include, 0=leave out)"))
        
        if swj == 1:
            dt = open("a452374.asc").read().split()
            for i in range(0,len(dt)):
                dt[i] = int(dt[i])
            dt = np.array(dt)
            dt = dt.reshape((10,np.shape(dt)[0]//10)).T     
            goto .dtstor
        else:
            goto .start
    
    
    elif loop == 8:
        loop = 9
        swj = int(input("Bus group 8: 1974 GMC model a5308 buses (12 buses total)\n(1=include, 0=leave out)"))
        
        if swj == 1:
            dt = open("a530874.asc").read().split()
            for i in range(0,len(dt)):
                dt[i] = int(dt[i])
            dt = np.array(dt)
            dt = dt.reshape((12,np.shape(dt)[0]//12)).T 
            goto .dtstor
        else:
            goto.start
    
    else:
        goto .fin
        
    
    #DISCRETIZE CONTINUOUS MILEAGE DATA
    
    label .dtstor
    
    
    ov1 = dt[5,:]
    ov2 = dt[8,:]
    nr = np.shape(dt)[0]
    m = np.shape(dt)[1]
    dtc = smu(b=sm(dt[11:nr,:]>=ov1), s=sm(ov1>0)) + smu(b=sm(dt[11:nr,:]>=ov2), s=sm(ov2>0))
    mil = dt[12:nr,:]-dt[11:nr-1,:]
    
    print("minimum, maximum, mean monthly mileage",min(np.min(mil,axis=0)),max(np.max(mil,axis=0)),np.round(np.mean(np.mean(mil,axis=0))) )
    print("begin discretizing data ... ")
    
    dtx = dt[11:nr,:]+ov1*dtc*(dtc-2)-0.5*ov2*dtc*(dtc-1)
    dtx = np.ceil(n*dtx/omax)
    dtc = np.concatenate([dtc[1:nr-11,:]-dtc[0:nr-12,:],np.zeros((1,m))], axis=0)
    mil = (dtx[1:nr-11,:]-dtx[0:nr-12,:])+dtx[0:nr-12,:]*dtc[0:nr-12,:]
    
    
    
    #COMPUTE NON-PARAMETRIC HAZARD ESTIMATE
    
    for j in range(1,n+1):
        dt = sm_(dtx == j)
        nc[(j-1),0] = np.array([np.sum((np.array([np.sum(dt,axis=0)]).T),axis=0)]).T
        rc[(j-1),0] = np.array([np.sum((np.array([np.sum(dt*dtc,axis=0)]).T),axis=0)]).T
        nt[(j-1),0] = nc[j-1,0] + nt[j-1,0]
        rt[(j-1),0] = rt[j-1,0] + rc[j-1,0]
        
    npk = rt/(nt-sm_(nt==0))
    
    vc = []
    for i in dtx.flatten(order='F'):
        vc.append(int(i)-1)
    
    lp = (1-npk)[vc,:]
    ll = ll + np.array([np.log(lp+(1-2*lp)*np.array([dtc.flatten(order='F')]).T).sum(axis=0)]).T
    
    #DISPLAY ESTIMATED DISCRETIZED MULTINOMIAL MILEAGE DISTRIBUTION
    
    print("minimum, maximum discretized mileage  {}  {}".format(np.min(np.min(mil, axis=0)),np.max(np.max(mil, axis=0)) ))
    
    if np.max(np.max(mil, axis=0)) > modnum[4]:
        
        modnum[4] = np.max(np.max(mil, axis=0))
    
    p = np.zeros((int(modnum[4])+1,1))
    for i in range(0,np.shape(p)[0]):
        p[i] = np.array([np.mean(np.reshape(sm_(mil==i).flatten(order='F'), newshape=(len(sm_(mil==1).flatten(order='F')), 1)),axis=0)]).T   
    
    
    #WRITE DISCRETIZED DATA TO GAUSS DATA FILE bdt.dat
    
    if rws > 0:
        goto .writerow
    
    #if modnum[7] == 0;                              UNNECESSARY PART
    #       create f1=bdt with bdt, 3, 2;
    #else;
    #    create f1=bdt with bdt, 4, 2;
    # endif;
    
    
    label .writerow
    
    if modnum[6] == 0:
        
        len_ = (nr-12) * m
        f1 = np.concatenate([np.reshape(np.array(dtc[1:nr-11,:].flatten(order='F')), (len_,1)), np.reshape(np.array(dtx[1:nr-11,:].flatten(order='F')), (len_,1)), np.reshape(np.array(mil.flatten(order='F')), (len_,1)) ], axis=1) 
        rws = len(f1)
        
    else:
        
        f1 = np.concatenate([dtc[1:nr-11,:], dtx[1:nr-11,:], mil, dtc[0:nr-12,:] ], axis=1)
        rws = len(f1)
    
    lfe = lfe + rws
    
    
    #PRINT SUMMARY OF ESTIMATED MULTINOMIAL MILEAGE DISTRIBUTION
    
    milecnt[0:np.shape(p)[0]] = milecnt[0:np.shape(p)[0]]+(nr-12)*m*p
    dataf = pd.DataFrame(data=np.concatenate([np.array([np.arange(start=0, stop= modnum[4]+1,step=1)]).T,p,milecnt[0:int(modnum[4])+1]/lfe ],axis=1), columns=["Mileage range", "current estimate", "cumulative estimate"], index=None)
    print(dataf)
    print("total rows written, cumulative, current", lfe, (nr-12)*m)
    
    goto .start
    
    label .fin
    
    #it says closeall here but idk if that's necessary
    p = milecnt[0:int(modnum[4])+1]/lfe
    ogrid = np.array([np.arange(start=0, stop= n,step=1)]).T * omax / (1000*n)
    
    print("STORDAT.GPR successfully created data file f1")
    print("ready to run SETUP.GPR to set parameters for NFXP algorithm")
    
    #GRAPH NON-PARAMETRIC HAZARD AND NUMBER OF OBSERVATIONS
    
    grf = int(input("Graph non-parametric estimate of replacement hazard? (1=yes,0=no) "))
    
    if grf:
        
        plt.figure(figsize=(12,10))
        plt.plot(ogrid, npk)
        plt.xlabel("Miles Since Last Replacement (000)")
        plt.ylabel("Replacment Probability")
        plt.title("Non-Parametric Hazard")
        plt.xticks(np.arange(min(ogrid), max(ogrid)+1, 20))
        plt.yticks(np.arange(min(npk),max(npk)+0.003,0.01))
        
        
        plt.figure(figsize=(12,10))
        plt.plot(ogrid,nt)
        plt.xlabel("Miles Since Last Replacement (000)")
        plt.ylabel("Number of Observations")
        plt.title("Replacement Observations")
        plt.xticks(np.arange(min(ogrid), max(ogrid)+1, 20))
        plt.yticks(np.arange(min(nt),max(nt)+100,200))
    
    

### display: displays menu of available cost function specifications

In [19]:
@with_goto
def display_():
    
    global dm,bet,modnum, q, qmax, qtol, qtol1
    
    if modnum[0] > 0:
        "cost function"
    else:
        modnum[0] = int(input("enter specification code for cost function or 0 to show menu ="))
        
    if modnum[1] == 1:
        goto .mod1
        
    elif modnum[1] == 2:
        goto .mod2
        
    elif  modnum[1] == 3:
        goto .mod3
    else:
        goto .mod4
        
    
    label .mod2
    
    if modnum[0] == 0:
        print("code 1: c(x)=.01*c1*x+.0001*c2*x^2")
        print("code 2: c(x)=.001*c1*x+.000001*c2*x^2")
        print("code 3: c(x)=c1*x^(.001*c2)")
        print("code 4: c(x)=c2/(91-x)+.01*c1*x^(.5)")
        print("code 5: c(x)=c2/((91-x)^(.5))+c1*x^(.5)")
        print("code 6: c(x)=c2/((91-x)^2)+c1*x^(.5)")
        print("code 7: c(x)=c1*exp(.001*c2*x)")
        modnum[0] = int(input("enter code number="))
    
    if modnum[0] == 0:
        goto .mod2
    elif modnum[0] == 1:
        cstr="c(x)=.01*c1*x+.0001*c2*x^2"
    elif modnum[0] == 2:
        cstr="c(x)=.001*c1*x+.000001*c2*x^2"
    elif modnum[0] == 3:
        cstr="c(x)=c1*x^(.001*c2)"
    elif modnum[0] == 4:
        cstr="c(x)=c2/(n+1-x)+.01*c1*x^(.5)"
    elif modnum[0] == 5:
        cstr="c(x)=c2/((n+1-x)^(.5))+c1*x^(.5)"
    elif modnum[0] == 6:
        cstr="c(x)=c2/((n+1-x)^2)+c1*x^(.5)"
    elif modnum[0] == 7:
        cstr="c(x)=c1*exp(.001*c2*x)"

    print(''.join(cstr))
    goto .modend
    
    
    label .mod1
    
    if modnum[0] == 0:
        print("code 1: c(x)=c1*x")
        print("code 2: c(x)=.001*c1*x")
        print("code 3: c(x)=c1*x^(.5)")
        print("code 4: c(x)=c1/(91-x)")
        print("code 5: c(x)=.001*c1*x^(.5)")
        modnum[0] = int(input("enter desired code="))
    
    if modnum[0] == 0:
        goto .mod1
    elif modnum[0] == 1:
        cstr="c(x)=c1*x"
    elif modnum[0] == 2:
        cstr="c(x)=.001*c1*x"
    elif modnum[0] == 3:
        cstr="c(x)=c1*x^(.5)"
    elif modnum[0] ==4:
        cstr="c(x)=c1/(n+1-x)"
    elif modnum[0] ==5:
        cstr="c(x)=.001*c1*x^(.5)"

    print(''.join(cstr))
    goto .modend
    
    
    label .mod3
    if modnum[0] ==0:
        print("code 1: c(x)=.01*c1*x+.0001*c2*x^2+.000001*c3*x^3")
        print("code 2: c(x)=linear spline with knots at 150k and 300k miles")
        modnum[0] = int(input("enter code number="))
    
    if modnum[0] == 0 :
        goto .mod3
    elif modnum[0] == 1 :
        cstr="c(x)=.01*c1*x+.0001*c2*x^2+.000001*c3*x^3"
    elif modnum[0] == 2:
        cstr="c(x)=linear spline with knots at 150k and 300k miles"
    
    print(''.join(cstr))
    goto .modend
    
    
    label .mod4
    
    if modnum[0] == 0:
        print("code 1: c(x)=.01*c1*x+.0001*c2*x^2+.000001*c3*x^3+.000000001*c4*x^4")
        print("code 2: c(x)=linear spline with knots at 100k, 200k, and 300k miles")
        modnum[0] = int(input("enter code number="))
    
    if modnum[0] == 0:
        print("GOTOMOD4")
    elif modnum[0] == 1:
        cstr="c(x)=.01*c1*x+.0001*c2*x^2+.000001*c3*x^3+.000000001*c4*x^4"
    elif modnum[0] == 2:
        cstr="c(x)=linear spline with knots at 100k, 200k, and 300k miles"
    
    print(''.join(cstr))
    goto .modend
    
    
    label .modend
    
    nvec = []
    if (modnum[[1,3]].flatten() == np.array([[1,0]])).all():
        dm = 2
        nvec = np.array([["tr"], ["c1"]])

    elif (modnum[[1,3]].flatten() == np.array([2,0])).all():
        dm = 3
        nvec = np.array([["tr"], ["c1"],["c2"]])

    elif (modnum[[1,3]].flatten() == np.array([3,0])).all():
        dm = 4
        nvec = np.array([["tr"], ["c1"],["c2"], ["c3"]])

    elif (modnum[[1,3]].flatten() == np.array([4,0])).all():
        dm = 5
        nvec = np.array([["tr"], ["c1"],["c2"], ["c3"],["c4"]])

    elif (modnum[[1,3]].flatten() == np.array([1,1])).all():
        dm = 3
        nvec = np.array([["tr"], ["c1"],["bet"]])

    elif (modnum[[1,3]].flatten() == np.array([2,1])).all():
        dm = 4
        nvec = np.array([["tr"], ["c1"],["c2"],["bet"]])

    elif (modnum[[1,3]].flatten() == np.array([3,1])).all():
        dm = 5
        nvec = np.array([["tr"], ["c1"],["c2"],["c3"],["bet"]])

    elif (modnum[[1,3]].flatten() == np.array([4,1])).all():
        dm = 6
        nvec = np.array([["tr"], ["c1"],["c2"],["c3"],["c4"],["bet"]])

    print("objective function parameters are:")
    for j in (nvec):
        print(j)
        
    if modnum[2] == 2:
   
        dm = dm + modnum[4]

        i = 1
        v = []

        while i <= modnum[4]:

                l = 'p{n}'.format(n=(i-1))
                v.append([l])
                i = i +1
                nvec = np.concatenate([nvec,v])


        print("mileage process parameters are   : {k}".format(k=nvec[dm-modnum[4]:dm,0]))
    
    if modnum[6] == 1:
        print("last coefficient is lagged control i(t-1)")
        dm = dm + 1 
        nvec = np.concatenate([nvec,np.array([["i(t-1)"]])])

    print('total dimension of parameter vector q= {}'.format(dm))
    
    return(modnum, nvec, cstr)



### setup: initialization routine for nested fixed point algorithm

In [20]:
@with_goto
def setup():
    
    global modnum, bet, q, qmax, qtol, qtol1, nvec, ss2,initmod,dm,maxstp, d, t, maxls, minls
    global itol1, itol2, ctol, hpr, pr, nstep, maxlfe, mincstp, maxcstp, ltol, stol, drvchk
    
    if 'modnum' in locals() or 'modnum' in globals():
        modnum = modnum
    else:
        modnum = np.array([0])
        
    if 'f1' not in locals() and 'f1' not in globals():
        print("Error: bdt.dat not found. Running STORDAT.GPR to create it")
        stordat()
    
    elif np.shape(modnum)[0] == 1:
        swj=int(input("Error: Discretization parameters set by STORDAT.GPR are not in memory\nLoad parameters from previous run of STORDAT.GPR? (1=yes, 0=no)"))   
    
        if swj:
            if 'n' not in locals() or 'n' not in globals():
                print("Error: grid size parameter from STORDAT.GPR not found on disk\nre-running STORDAT.GPR")
                stordat()
                
        else:
            print("rerunning STORDAT.GPR to correctly initialize data and parameters")
            stordat()
            
    
    #INITIALIZE VARIABLES  if program hasn't been previously run
    
    if modnum[1]==0:
        print("Parameters of NFXP algorithm were not all previously specified\nNew values will be initialized and can be changed below")
      
        initmod = 1
        bet = 0.9999
        dm = 2
        q = np.array([[11],[3]])
        qmax = np.array([[1000],[1000]])
        qtol = np.array([[0.01],[0.01]])
        qtol1 = np.array([[10],[10]])
        maxstp = 20
        modnum[0:4] = np.array([[2],[1],[1],[0]])
        d = 0.5
        t = 500
        maxls = 6
        minls = 0
        ss2 = 1
        itol1 = 0.999
        itol2 = 10**(-12)
        ctol=10**(-12)
        hpr=1
        pr=1
        nstep=10
        maxlfe=200
        mincstp=2
        maxcstp=100
        ltol=10**(-5)
        stol=1
        drvchk=0
        
    else:
        initmod=0

    
    #PRINT OUT EXISTING VARIABLE SETTINGS, and change if desired
    
    print("fixed point dimension =",n)
    
    if modnum[2] == 1:
        print("estimating partial likelihood function P(i|x)")
    else:
        print("estimating full likelihood function P(i|x)p(x|x(-1),i(-1))")
    
    print("maximum value of [x(t)-x(t-1)] in data set =",int(modnum[4]))
    
    if modnum[3] == 0:
        print("beta parameter fixed at bet =",bet)
    else:
        print("initial estimate of beta parameter =",bet)
    
    if modnum[0] <= 0:
        modnum[0] = 2
    
    dsp_ = display_()
    modnum = dsp_[0]
    nvec = dsp_[1]
    cstr = dsp_[2]
    
    
    label .check
    
    swj = int(input("are these the settings you want? (1=yes 0=no)"))
    
    if swj == 0:
        
        initmod = 0
        modnum[2] = int(input("enter 1 if you wish to estimate partial likelihood P(i|x)\nenter 2 if you wish to estimate full likelihood P(i|x)p(x|x(-1),i(-1))"))
        modnum[3] = int(input("do you want to estimate discount factor `bet'? (1=yes, 0=no)"))
        
        if modnum[3]==0:
            bet=int(input("beta fixed: enter value 0<bet<1"))
        
        modnum[1] = int(input("enter desired number of parameters for cost function c(x): 1,2,3,4:"))
        modnum[0] = 0
        dsp1 = display_()
        modnum = dsp1[0]
        nvec = dsp1[1]
        cstr = dsp1[2]
        
        goto .check
    
    
    #display, change nfxp algorithm settings
    
    print("Settings for NFXP algorithm:")
    
    label .setting
    
    print("Jacobian update factor d, 0<d<1 for fixed point algorithm")
    print("d=1 corresponds to Newton-Kantorovich method")
    print("d=0 corresponds to Stirling's method")
    print("d=.5 corresponds to Werner's method")
    print("Jacobian update factor d, 0<d<1 ", d)
    print("rows of data set BDT.DAT read per pass ", t)
    print("maximum number of lf evaluations ", maxlfe)
    print("maximum stepsize ", maxstp)
    print("maximum linesearch iterations ", maxls)
    print("minimum linesearch iterations ", minls)
    print("maximum contraction iterations", maxcstp)
    print("minimum contraction iterations ", mincstp)
    print("maximum number of NK iterations ", nstep)
    print("switch to step size 1 when grad*direc < ", itol2)
    print("switch to BFGS when grad*direc < ", stol)
    print("lf ratio to terminate line search ", itol1)
    print("initial stepsize ", ss2)
    print("outer convergence tolerance ", ctol)
    print("inner convergence tolerance ", ltol)
    print("check derivatives of lf? (1=yes, 0=no) ", drvchk)
    print("print hessian matrix?  (1=yes, 0=no) ", hpr)
    print("print fixed point info? (1=yes, 0=no) ", pr)
    
    
    #PROMPT USER FOR NEW SETTINGS
    
    swj = int(input("are these settings OK? (1=yes, 0=no)"))
    
    if swj == 0:
        
        print("enter new parameter values")
        d = int(input("Jacobian update factor d, 0<d<1"))
        t = int(input("rows of data set read per pass"))
        maxlfe = int(input("maximum number of lf evaluations"))
        maxstp = int(input("maximum stepsize"))
        maxls = int(input("maximum linesearch iterations"))
        minls = int(input("minimum linesearch iterations"))
        maxcstp = int(input("maximum contraction iterations"))
        mincstp = int(input("minimum contraction iterations"))
        nstep = int(input("maximum number of NK iterations"))
        itol2 = int(input("switch to step size 1 if grad*direc <"))
        stol = int(input("switch to BFGS when grad*direc <"))
        itol1 = int(input("lf ratio to terminate line search"))
        ss2 = int(input("initial stepsize"))
        ctol = int(input("outer convergence tolerance"))
        ltol = int(input("inner convergence tolerance"))
        drvchk = int(input("check derivatives of lf? (1=yes, 0=no)"))
        hpr = int(input("print hessian matrix?   (1=yes, 0=no)"))
        pr = int(input("print fixed point info? (1=yes, 0=no)"))
        
        goto .setting
    
    
    #DISPLAY AND SELECT INITIAL PARAMETER ESTIMATES
    
    if modnum[2] == 2:
        
        q[(dm-int(modnum[4])-int(modnum[6])):(dm-int(modnum[6])),0] = p[0:int(modnum[4]),0]
        
    
    swj = int(input("enter 1 to input starting values for q or 0 to use current values"))
    
    
    label .redo
    
    if swj == 1:
        
        print("Parameter vector q (enter letter 'l' to leave as is)")
    
        label .qinpt
        
        print(list(nvec.flatten()))
        print(q.T)
        
        a1 = int(input("are current values OK? (1=yes,0=no) "))

        if a1 == 0:

            for i in range(1,len(nvec)+1):
                q[i-1]=int(input("Current value for {} is = {}. Please enter new value = ".format(list(nvec.flatten())[i-1],int(q[i-1]))))
        
            goto .qinpt
        
        print("absolute parameter bounds (enter 'l' to leave as is) ")
        
        qmax = 100 * np.ones((np.shape(q)[0],1))
        
        
        label .qmaxinpt
        
        print(list(nvec.flatten()))
        print(qmax.T)
        
        a1 = int(input("are current values OK? (1=yes,0=no) "))
        
        if a1 == 0:

            for i in range(1,len(nvec)+1):
                qmax[i-1]=int(input("Current value for {} is = {}. Please enter new value = ".format(list(nvec.flatten())[i-1],int(qmax[i-1]))))
        
            goto .qmaxinpt
        
        print("Set minimum contraction steps if q changes less than ('l' to leave as is)")
        qtol = 0.1 * np.ones((np.shape(q)[0],1))
        
        
        label .qtolinpt
        
        print(list(nvec.flatten()))
        print(qtol.T)
        
        a1 = int(input("are current values OK? (1=yes,0=no) "))
        
        if a1 == 0:

            for i in range(1,len(nvec)+1):
                qtol[i-1]=int(input("Current value for {} is = {}. Please enter new value = ".format(list(nvec.flatten())[i-1],int(qtol[i-1]))))
        
            goto .qtolinpt
        
        print("Set maximum contraction steps if q changes more than ('l' to leave as is)")
        qtol1 = 0.1 * np.ones((np.shape(q)[0],1))
        
        
        label .qtl1inpt
        
        
        print(list(nvec.flatten()))
        print(qtol1.T)
        
        a1 = int(input("are current values OK? (1=yes,0=no) "))
        
        if a1 == 0:

            for i in range(1,len(nvec)+1):
                qtol1[i-1]=int(input("Current value for {} is = {}. Please enter new value = ".format(list(nvec.flatten())[i-1],int(qtol1[i-1]))))
        
            goto .qtl1inpt
        

    
    print(list(nvec.flatten()), list(q.flatten()))
    print(list(nvec.flatten()), list(qmax.flatten()))
    print(list(nvec.flatten()), list(qtol.flatten()))
    print(list(nvec.flatten()), list(qtol1.flatten()))
    

    if modnum[3]==0:

        print("beta not estimated. Its value is fixed at  :{}".format(bet))

    else:

        if(modnum[[2,3]].flatten() == np.array([2,1])).all():

                print("beta is estimated. Its initial value is  :{}".format(float(1/(1+np.exp(q[dm-modnum[4]-modnum[6],0])))))


        elif(modnum[[2,3]].flatten() == np.array([1,1])).all():

                print("beta is estimated. Its initial value is  :{}".format(float(1/(1+np.exp(q[dm-modnum[6]-1,0])))))


    swj = int(input("are these the values you want? (1=yes, 0=no) "))
    
    if swj == 0:
        swj = 1
        goto .redo
    
    print("SETUP complete: ready to run NFXP.GPR")
    
    label .bye

### Run the cell below as well.

In [21]:
fmt = np.array([["-*.*lf", 6, 0],
             ["-*.*s", 8, 8],
             ["*.*lf",14, 6],
             ["*.*lf", 14, 6],
             ["*.*lf",14, 6],
             ["*.*lf",14, 6]])
if 'modnum' in locals() or 'modnum' in globals():
    modnum = modnum
else:
    modnum = np.array([0])

### nfxp: nested fixed point maximum likelihood algorithm

In [102]:
@with_goto
def nfxp():

    global bet, rtol, drvchk, delta, oll, nr, swj, err, pass_, lp, dt, pk, nstep, c
    global mincstp, q11, q22, tt, h, dll, ll, ii, dt, f1, h, q0, ndll, dfs, idf0
    global q, ndll, qk, rk, nwt, ftol, ls, lfe, ll, lcstp, dm, direc, dparm
    global modnum, nvec, cstr, ss2, ss1, cstp, maxcstp, tcstp, ev, ttt,i, ev1, pr,nt
    global m
    global j,tt,nwt,tol,tol1,mm,ev,ev1, pr, swj, pk, dev
    
    #DISPLAY TOLERANCE AND INITIAL PARAMETER SETTINGS
    
    label .setup

    if np.shape(modnum)[0] == 1 or modnum[0] == 0:
        #INCLUDE SETUP.GPR HERE
        if swj < 0:
            goto .abort

    modnum = display_()[0]
    nvec = display_()[1]
    cstr = display_()[2]

    print("fixed point dimension n={}".format(n))

    if ss2 == 0:
        ss2 = 1

    label .settings
    
    if modnum[3] == 0:
        
        print("beta value, 0<bet<1  {}".format(bet))
    
    if modnum[2] == 1:
        
        print("transition probabilities {}".format(p[0:1+int(modnum[4]),0].T))
    
    print("jacobian index, d (0<d<1) {}".format(d))
    print("maximum number of lf evaluations {}".format(maxlfe))
    print("maximum number of NK steps  {}".format(nstep))
    print("maximum stepsize {}".format(maxstp))
    print("maximum linesearch iterations {} ".format(maxls))
    print("minimum linesearch iterations {}".format(minls))
    print("maximum contraction iterations {}".format(maxcstp))
    print("minimum contraction iterations {}".format(mincstp))
    print("switch to step size 1 when grad*direc< {}".format(itol2))
    print("lf ratio to terminate line search {}".format(itol1))
    print("initial stepsize {}".format(ss2))
    print("outer convergence tolerance {}".format(ctol))
    print("inner convergence tolerance {}".format(ltol))
    print("switch to Broyden when grad*direc < {}".format(stol))
    print("check derivatives of lf? (1=yes, 0=no) {}".format(drvchk))
    print("print hessian matrix?  (1=yes, 0=no) {}".format(hpr))
    print("print fixed point info? (1=yes, 0=no) {}".format(pr))
    swj = int(input("are these settings OK? (1=yes, 0=no)  "))
    
    if swj == 0:
        print("Run  SETUP.GPR to change settings")
        goto .setup
     
    if bet > 0:
        cstp = maxcstp
        tcstp = cstp
        ev = np.zeros((n,1))
        print("set cstp=maxcstp when (q-q0) > : {}".format(qtol1.T))
        print("set cstp=mincstp when (q-q0) < : {}".format(qtol.T))
    
    ev = np.zeros((n,1))
    
    print("absolute parameter bounds {}".format(qmax.T))
    print("initial parameter estimates {}".format(q.T))
    
    if (modnum[[2,3]].flatten() == np.array([[2,1]])).all():
        
        print("(implied beta value){}".format(1/(1+np.exp(q[dm-int(modnum[4])-1,0]))))
    
    elif (modnum[[2,3]].flatten() == np.array([[1,1]])).all():
        
        print("(implied beta value){}".format(1/(1+np.exp(q[dm-1,0]))))
    
    print("number of rows read per pass {}.".format(t))
    
    grf = int(input("plot hazard and value functions? (1=yes,0=no)  "))
    
    if grf:
        
        plt.figure(figsize=(12,10))
        plt.plot(ogrid, npk)
        plt.xlabel("Miles Since Last Replacement (000)")
        plt.ylabel("Replacment Probability")
        plt.title("Non-Parametric Hazard")
        plt.xticks(np.arange(min(ogrid), max(ogrid)+1, 20))
        plt.yticks(np.arange(min(npk),max(npk)+0.003,0.01))
        
        
        plt.figure(figsize=(12,10))
        plt.plot(ogrid,nt)
        plt.xlabel("Miles Since Last Replacement (000)")
        plt.ylabel("Number of Observations")
        plt.title("Replacement Observations")
        plt.xticks(np.arange(min(ogrid), max(ogrid)+1, 20))
        plt.yticks(np.arange(min(nt),max(nt)+100,200))
        
        
    #INITIALIZE VARIABLES USED IN PROGRAM
    
    ttt = time.time()
    i = 1
    ss1 = 0
    nwt = 0
    ftol = 1
    ls = 0
    lfe = 0
    ll = 0
    idf0 = 2*stol
    m = math.floor(n/2)
    q11 = np.zeros((m,m+int(modnum[4])))
    q22 = np.zeros((n-m,n-m))
    q0 = np.zeros((dm,1))
    lcstp = cstp
    dm = np.shape(q)[0]
    direc = np.ones((dm,1))
    dparm = np.ones((dm,1))
    
    if bet < 0.98:
        rtol=0.0001
    else:
        rtol=0.0000001 
    
    if drvchk:
        
        delta = int(input("Enter delta value for numerical derivatives  "))
        oll=0
    
    
    
    
    #OPEN FILE BDT.DAT FOR READ
    
    nr = np.shape(f1)[0]
    swj = 0
    err = 0
    print("Number of rows of data{}".format(nr))
    pass_ = (nr-(nr%t))/t
    nr = nr%t
    
    
    #BEGIN OUTER BHHH OPTIMIZATION ALGORITHM
    
    label .bhhhstart
    
    lfe = lfe + 1
    
    if lfe >= maxlfe:
        
        print("terminate at lf evaluation limit {}".format(maxlfe))
        goto .term
    
    if ls > 0:
        pr = 0
        
    if ls == 1:
        cstp = lcstp
    
    
    #UPDATE COST FUNCTION c FOR CURRENT PARAMETER ESTIMATES
    
    function_()
    d1v = 0
    lp = 0
    dt = 0
    
    if bet == 0:
        
        pk = 1/(1+np.exp(c-tr-c[0,0]))
        goto .core
    
    
    #ADAPTIVE DETERMINATION OF SWITCHPOINT FROM CONTRACTION TO Newton-Kantorovich iterations
    
    if(np.sum(sm(abs(dparm)>qtol1),axis=0)>0):
        tcstp = maxcstp
    
    if((abs(dparm)<qtol).all()):
        tcstp = mincstp
        
        if (ftol < ltol):
            nstep = 1
        
        elif (ftol>=ltol):
            nstep = 4
    
    elif (ftol>ltol and nstep ==1):
        nstep = 4
      
    if (lfe>1 and (ftol>ltol)):
        tcstp = cstp + 10
        
    if (nwt>2 and swj>=cstp):
        tcstp = cstp + 10
    
    elif (nwt>2 and swj<cstp):
        rtol = 0.5*rtol
    
    elif (ftol<ltol and swj>=cstp):
        tcstp = cstp-10
    
    elif(ftol<ltol and swj<cstp):
        rtol = 2*rtol
    
    cstp = tcstp
    
    if cstp>maxcstp:
        cstp = maxcstp
    
    elif cstp<mincstp:
        cstp = mincstp
    
    #UPDATE TRANSITION MATRIX P FOR CURRENT PARAMETER ESTIMATES
    
    if not(modnum[2]==1 and i >1):
        
        eq11()
        eq22()
        
    
    #CALL INNER FIXED POINT ALGORITHM
    
    ffxp()
    
    if pr:
        print('return to outer BHHH algorithm')
    
    #CORE OF BHHH ALGORITHM: CALCULATION OF DIRECTION AND STEPSIZE
    
    label .core
    
    tt = time.time()
    h = np.zeros((dm,dm))
    dll = np.zeros((dm,1))
    ll = 0
    ii = math.ceil(nr/t)
    
    #read in a chunk of data into the matrix dt
    dt = f1
    
    if ii>pass_:
        nt = nr
    else:
        nt = t
    
    #EVALUATE LOGLIKELIHOOD FUNCTION AND DERIVATIVES for data in dt 
    #call eval if no lagged dependent variable, otherwise call eval1
    
    if modnum[6] == 0:
        eval_()
    
    else:
        eval1_()
    
    #CUMULATE MOMENT MATRIX 
    
    if ls == 0:
        
        h = h + np.einsum('ij,ik->jk',d1v,d1v)
    
    #y=seekr(f1,1);  /* reposition pointer at beginning of data file for next call to evaluate the likelihood */
    
    #Conduct derivative check if drvchk = 1
    
    if drvchk:
        
        if(i==1):
            
            q0 = q
            ndll = np.zeros((dm,1))
            oll = ll
            print("Numerical check of derivatives at the followin initial parameters")
            
            dfs = np.concatenate([nvec, q, direc, dll, (q/np.sqrt(np.array([np.diagonal(k)]).T))], axis=1)
            print(pd.DataFrame(data=dfs, columns=["Parameter", "Estimates", "direction","gradient","t-stat"]))
            
            if (modnum[[2,3]].flatten() == np.array([2,1])).all():
                print("implied bet value{}".format(1/(1+np.exp(q[dm-int(modnum[4])-1,0]))))
            
            if (modnum[[2,3]].flatten() == np.array([1,1])).all():
                print("implied bet value{}".format(1/(1+np.exp(dm-1,0))))
            
            print("log likelihood ss=0   {}".format(ll))
            
            idf0 = dll.T * direc
            print("grad*direc {}".format(idf0))
            
            print("Iteration {}".format(i))
            print("evaluation number  {}".format(lfe))
            
        
        elif((i>1) and (i<=dm+1)):
            
            q[i-1] = q0[i-1]
            ndll[i-1] = (ll-oll)/delta
        
        else:
            
            print("Comparison of analytical and numerical gradients with delta=  {}".format(delta))
            dfu = np.concatenate([nvec, q, dll, ndll, abs(dll-ndll)])
            print(pd.DataFrame(data=dfu, columns=["Parameter","Value","analytic","numerical", "difference"]))
        
            goto .abort
        
        
        if(i<=dm):
            
            q[i] = q0[i] + delta
            i = i + 1
            goto .bhhhstart
        
        else:
            
            i = i + 1
            goto .bhhhstart
    
    
    
    #MEMORIZE NUMBER OF CONTRACTION STEPS, BRANCH TO LINE SEARCH
    
    if ls == 1:
        lcstp=cstp
    
    if ls > 0:
        goto .lsrch
    
    #CALCULATE NEW DIRECTION VECTOR, CHECK FOR NUMERICAL ERRORS
    
    if hpr > 0:
        
        print("Cumulated outer product of first derivatives {}".format(h))
    
    
    if idf0 > stol: #start out using BHHH 
    
        h = np.linalg.inv(h)
        h0 = h
    
    else: #update h recursively using BFGS iterations
        
        qk =(dll0-dll)
        rk = ss2 * direc
        
        if np.dot(rk.T*qk)>0:
            print("using BFGS update for h ")
            h = (h-np.dot(np.dot(h,qk),np.dot(qk.T,h)) / np.dot(np.dot(qk.T,h),qk))*(np.dot(rk.T,qk))/(np.dot(np.dot(qk.T,h),qk))+np.dot(rk,rk.T)/np.dot(rk.T,qk)
            
            #Davidon-Fletcher-Powell update formula is h=h+rk*(rk')/(rk'rk)-h*qk*qk'h/(qk'h*qk)
            
        else:
            print("BFGS failed: using last h  {}".format(np.dot(rk.T,qk)))
            h0 = h
            
    
    if hpr > 0:
        print(h)
        
    direc = h*dll
    
    if ss2>2:
        ss2=1
    
    q0 = q
    
    q = q0 + ss2 * direc  #update new parameter vector as previous value time the stepsize ss2 times the direction vector
    
    tt = time.time() - tt
    
    if abs(q) >= qmax:
        
        ss2 = np.min(0.9 * qmax/abs(q0+direc),axis=0)
        q = q0+ss2*direc
        
        print("parameter out of bounds ss = {}".format(ss2))
    
    
    #PRINT OUT SUMMARY OF CURRENT ITERATION 
    
    print(" Parameter estimates for cost function ", cstr)
    
    dfm = np.concatenate([nvec, q, direc, dll, (q/np.sqrt(np.array([np.diagonal(k)]).T))], axis=1)
    print(pd.DataFrame(data=dfm, columns=["Parameter","Estimates","direction","gradient","t-stat"]))
    
    
    if (modnum[[2,3]].flatten() == np.array([[2,1]])).all():
        print("implied bet value  {}".format(1/(1+np.exp(q[dm-int(modnum[4])-1,0]))))
            
    elif (modnum[[2,3]].flatten() == np.array([[1,1]])).all():
        print("implied bet value  {}".format(1/(1+np.exp(q[dm-1,0]))))
        
    idf0 = np.dot(dll.T,direc)
    
    print("log likelihood ss=0 ", ll)
    print("grad*direc", idf0)
    print("Iteration", i)
    print("evaluation number", lfe)
    print("Cpu time to cumulate moment matrix ", tt)
    
    i = i + 1  #increment iteration counter
    
    if idf0 < ctol:
        goto .term
    
    elif idf0 < itol2:
        ls=0
        ss2=1
        lcstp=cstp
    
    else:
        ss0=0
        ss1=ss2
        df0 = idf0
        dll0 = dll
        lll = ll
        ls=1
        dparm = ss2 * direc
    
    if grf:
        delay0 = time.time()
        delay1 = time.time()
        
        while((delay1-delay0)/100 <= 2):
            delay1 = time.time()
    
    
        plt.figure(figsize=(12,10))
        plt.plot(ogrid,(1-pk),label='DP Model Hazard')
        plt.plot(ogrid,npk,label='Non-Parametric Hazard')
        plt.legend()
        plt.ylabel("Probability of Replacement")
        plt.title("Estimated Hazard Functions: NFXP Iteration {}".format(i))

        plt.figure(figsize=(12,10))
        plt.plot(ogrid,(c-c[0]),label='c')
        plt.plot(ogrid,(-bet*ev+bet*ev[0]),label='ß')
        plt.plot(ogrid,(c-bet*ev-c[0]+bet*ev[0]),label='c+ß')
        plt.legend()
        plt.ylabel("Normalized Cost/Value Functions")
        plt.title("Estimated Cost/Value Functions: NFXP Iteration {}".format(i))
    
    
    goto .bhhhstart
    
    
    #LINESEARCH ALGORITHM BY SECANT METHOD  
    
    label .lsrch
    
    print("linesearch  {}".format(ls))
    
    print("derivative time  {}".format(time.time()-tt))
    
    print("likelihood ss  ",ss2, ll)
    
    df1 = np.dot(dll.T,direc)
    
    print("df1 = {}, df0 = {}".format(df1,df0))
    
    ss2 = ss1-df1*(ss0-ss1)/(df0-df1)
    
    if ss2 < 0:
        
        print("negative step", ss2)
        ss2 = ss1
        ls = 0
        q = q0 + ss2 * direc
        dparm = np.zeros((dm,1))
        goto .bhhhstart
    
    if ss2 > maxstp:
        
        print("step exceeds maximum", ss2)
        ss2 = ss1
        ls = 0
        q = q0 + ss2 * direc
        dparm = np.zeros((dm,1))
        goto .bhhhstart
    
    if ls > maxls:
        
        print("line search limit ss = {}",ss2)
        ls = 0
        q = q0 + ss2 * direc
        dparm = (ss2-ss1) * direc
        goto .bhhhstart
    
    if(ll/lll > itol1 and ls > minls) or ss2 > maxstp:
        
        print("last step ss = {}".format(ss2))
        ls = 0
        q = q0 + ss2 * direc
        dparm = (ss2 - ss1) *direc
        goto .bhhhstart
        
    q = q0 + ss2 * direc
    dparm = (ss2-ss1)*direc
    ls = ls + 1
    df0 = df1
    lll = ll
    ss0 = ss1
    ss1 = ss2
    
    goto .bhhhstart
    
    #TERMINATION OF PROGRAM
    
    label .term
    
    print("NFXP converged: total execution time  {} seconds".format(time.time()-ttt))
    
    label .abort
    
    
    

In [140]:
stordat()

enter desired fixed point dimension 'n' 90
enter upper bound odometer value (recommend 450,000)450000
implied size of discrete mileage range=5000.0
include lagged replacement choice i(t-1) in data set? (1=yes, 0=no)0
enter bus groups to be included in BDT.DAT (1=include, 0=leave out)
Bus group 1: 1983 Grumman model 870 buses (15 buses total)
(1=include, 0=leave out)0
Bus group 2: 1981 Chance RT-50 buses (4 buses total)
(1=include, 0=leave out)0
Bus group 3: 1979 GMC model t8h203 buses (48 buses total)
(1=include, 0=leave out)0
Bus group 4: 1975 GMC model a5308 buses (37 buses total)
(1=include, 0=leave out)0
Bus group 5: 1972 GMC model a5308 buses (18 buses total)
(1=include, 0=leave out)0
Bus group 6: 1972 GMC model a4523 buses (18 buses total)
(1=include, 0=leave out)0
Bus group 7: 1974 GMC model a4523 buses (10 buses total)
(1=include, 0=leave out)0
Bus group 8: 1974 GMC model a5308 buses (12 buses total)
(1=include, 0=leave out)1
minimum, maximum, mean monthly mileage 0 7134 2568.0

In [141]:
setup()

Parameters of NFXP algorithm were not all previously specified
New values will be initialized and can be changed below
fixed point dimension = 90
estimating partial likelihood function P(i|x)
maximum value of [x(t)-x(t-1)] in data set = 2
beta parameter fixed at bet = 0.9999
c(x)=.001*c1*x
objective function parameters are:
['tr']
['c1']
total dimension of parameter vector q= 2
are these the settings you want? (1=yes 0=no)1
Settings for NFXP algorithm:
Jacobian update factor d, 0<d<1 for fixed point algorithm
d=1 corresponds to Newton-Kantorovich method
d=0 corresponds to Stirling's method
d=.5 corresponds to Werner's method
Jacobian update factor d, 0<d<1  0.5
rows of data set BDT.DAT read per pass  500
maximum number of lf evaluations  200
maximum stepsize  20
maximum linesearch iterations  6
minimum linesearch iterations  0
maximum contraction iterations 100
minimum contraction iterations  2
maximum number of NK iterations  10
switch to step size 1 when grad*direc <  1e-12
switch to

In [142]:
nfxp()

c(x)=.001*c1*x
objective function parameters are:
['tr']
['c1']
total dimension of parameter vector q= 2
c(x)=.001*c1*x
objective function parameters are:
['tr']
['c1']
total dimension of parameter vector q= 2
c(x)=.001*c1*x
objective function parameters are:
['tr']
['c1']
total dimension of parameter vector q= 2
fixed point dimension n=90
beta value, 0<bet<1  0.9999
transition probabilities [0.48866667 0.50666667 0.00466667]
jacobian index, d (0<d<1) 0.5
maximum number of lf evaluations 200
maximum number of NK steps  10
maximum stepsize 20
maximum linesearch iterations 6 
minimum linesearch iterations 0
maximum contraction iterations 100
minimum contraction iterations 2
switch to step size 1 when grad*direc< 1e-12
lf ratio to terminate line search 0.999
initial stepsize 1
outer convergence tolerance 1e-12
inner convergence tolerance 1e-05
switch to Broyden when grad*direc < 1
check derivatives of lf? (1=yes, 0=no) 0
print hessian matrix?  (1=yes, 0=no) 1
print fixed point info? (1=ye

C:\Users\90538\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  # This is added back by InteractiveShellApp.init_path()
C:\Users\90538\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: ComplexWarning: Casting complex values to real discards the imaginary part


ValueError: math domain error

In [31]:
program()

enter desired fixed point dimension 'n' 90
enter upper bound odometer value (recommend 450,000)450000
implied size of discrete mileage range=5000.0
include lagged replacement choice i(t-1) in data set? (1=yes, 0=no)0
enter bus groups to be included in BDT.DAT (1=include, 0=leave out)
Bus group 1: 1983 Grumman model 870 buses (15 buses total)
(1=include, 0=leave out)0
Bus group 2: 1981 Chance RT-50 buses (4 buses total)
(1=include, 0=leave out)0
Bus group 3: 1979 GMC model t8h203 buses (48 buses total)
(1=include, 0=leave out)0
Bus group 4: 1975 GMC model a5308 buses (37 buses total)
(1=include, 0=leave out)0
Bus group 5: 1972 GMC model a5308 buses (18 buses total)
(1=include, 0=leave out)0
Bus group 6: 1972 GMC model a4523 buses (18 buses total)
(1=include, 0=leave out)0
Bus group 7: 1974 GMC model a4523 buses (10 buses total)
(1=include, 0=leave out)0
Bus group 8: 1974 GMC model a5308 buses (12 buses total)
(1=include, 0=leave out)1
minimum, maximum, mean monthly mileage 0 7134 2568.0

C:\Users\90538\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
C:\Users\90538\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: RuntimeWarning: invalid value encountered in add
C:\Users\90538\Anaconda3\lib\site-packages\ipykernel_launcher.py:113: RuntimeWarning: invalid value encountered in less
C:\Users\90538\Anaconda3\lib\site-packages\ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in less


ValueError: could not broadcast input array from shape (90) into shape (1)

##  TRY

## Aşağıda yazığım kod Nested Fixed Point Algorithm'in açılmış versiyonu. Nerede hata olduğunu görebilmek için açık açık yazdım.

In [ ]:
    modnum = display_()[0]
    nvec = display_()[1]
    cstr = display_()[2]


In [ ]:
    if ss2 == 0:
        ss2 = 1

In [ ]:
    if bet > 0:
        cstp = maxcstp
        tcstp = cstp
        ev = np.zeros((n,1))

In [ ]:
    ttt = time.time()
    i = 1
    ss1 = 0
    nwt = 0
    ftol = 1
    ls = 0
    lfe = 0
    ll = 0
    idf0 = 2*stol
    m = math.floor(n/2)
    q11 = np.zeros((m,m+int(modnum[4])))
    q22 = np.zeros((n-m,n-m))
    q0 = np.zeros((dm,1))
    lcstp = cstp
    dm = np.shape(q)[0]
    direc = np.ones((dm,1))
    dparm = np.ones((dm,1))

In [ ]:
    if bet < 0.98:
        rtol=0.0001
    else:
        rtol=0.0000001 

In [ ]:
    nr = np.shape(f1)[0]
    swj = 0
    err = 0

In [ ]:
    pass_ = (nr-(nr%t))/t
    nr = nr%t

In [ ]:
 lfe = lfe + 1

In [ ]:
    function_()
    d1v = 0
    lp = 0
    dt = 0
    

In [ ]:
    if(np.sum(sm(abs(dparm)>qtol1),axis=0)>0):
        tcstp = maxcstp
    
    if((abs(dparm)<qtol).all()):
        tcstp = mincstp
        
        if (ftol < ltol):
            nstep = 1
        
        elif (ftol>=ltol):
            nstep = 4
    
    elif (ftol>ltol and nstep ==1):
        nstep = 4
      
    if (lfe>1 and (ftol>ltol)):
        tcstp = cstp + 10
        
    if (nwt>2 and swj>=cstp):
        tcstp = cstp + 10
    
    elif (nwt>2 and swj<cstp):
        rtol = 0.5*rtol
    
    elif (ftol<ltol and swj>=cstp):
        tcstp = cstp-10
    
    elif(ftol<ltol and swj<cstp):
        rtol = 2*rtol
    
    cstp = tcstp
    
    if cstp>maxcstp:
        cstp = maxcstp
    
    elif cstp<mincstp:
        cstp = mincstp

In [ ]:
    if not(modnum[2]==1 and i >1):
        
        eq11()
        eq22()

In [ ]:
    nwt = 0
    j = 0 
    tol1 = 1

In [ ]:
j = j +2 

In [ ]:
ev1 = contract(ev)

In [ ]:
ev = contract(ev1)

In [ ]:
    tol = np.array([np.max(abs(ev-ev1), axis=0)]).T
    mm = tol/tol1
    tol1 = tol

In [ ]:
    nwt = 1
    swj = j

In [ ]:
ev1 = contract(ev)

In [ ]:
    pk = 1/(1+np.exp(c-bet*(d*ev+(1-d)*ev1)-tr-c[0,0]+bet*(d*ev[0,0]+(1-d)*ev1[0,0])))


In [ ]:
ev1=ev-partsolv(ev-ev1)

In [ ]:
tol = np.array([np.max(abs(ev-ev1), axis=0)]).T

In [ ]:
tol

In [ ]:
ev=contract(ev1)

In [ ]:
tol = np.array([np.max(abs(ev-ev1), axis=0)]).T

In [ ]:
tol

In [ ]:
nwt = nwt + 1

In [ ]:
pk=1/(1+np.exp(c-bet*(d*ev1+(1-d)*ev)-tr-c[0,0]+bet*(d*ev1[0,0]+(1-d)*ev[0,0])))


In [ ]:
ev=ev1-partsolv((ev1-ev))

In [ ]:
tol = np.array([np.max(abs(ev-ev1), axis=0)]).T

In [ ]:
tol

In [ ]:
ev1 = contract(ev)

In [ ]:
tol = np.array([np.max(abs(ev-ev1), axis=0)]).T

In [ ]:
tol

In [ ]:
nwt = nwt + 1

In [ ]:
pk = 1/(1+np.exp(c-bet*(d*ev+(1-d)*ev1)-tr-c[0,0]+bet*(d*ev[0,0]+(1-d)*ev1[0,0])))


In [ ]:
dev = np.concatenate([(e(pk)-1), (e(((dc[0,:]-dc)*pk))-dc[0,:])], axis=1)


In [ ]:
dev = np.concatenate([dev, (ev-ev1)], axis=1)

In [ ]:
dev = partsolv(dev)

In [ ]:
ev1 = ev-np.array([dev[:,dm-int(modnum[6])]]).T

In [ ]:
tol = np.array([np.max(abs(ev-ev1), axis=0)]).T

In [ ]:
tol

In [ ]:
dev[:,0:dm-int(modnum[6])] = bet*dev[:,0:dm-int(modnum[6])]

In [ ]:
ev = contract(ev1)

In [ ]:
pk = 1/(1+np.exp(c-bet*ev-tr-c[0,0]+bet*ev[0,0]))

In [ ]:
tol = np.array([np.max(abs(ev-ev1), axis=0)]).T

In [ ]:
tol
